<a href="https://colab.research.google.com/github/fabianhuertas1992/Prefactibilidad_SUAN/blob/main/Script/An%C3%A1lisis_Coberturas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Acceso a drive - Instalar librerias**

In [6]:
# @title Montar acceso Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# @title Librerias
!pip install rasterio
!pip install geopandas

# Análisis Coberturas **NDVI**

In [8]:
# @title Rutas de las carpetas con imágenes anteriores y posteriores

# Definir las carpetas de imágenes
carpeta_imagenes_1 = "/content/drive/MyDrive/imagenes_pruebas_Angie/imagenes_salida/choco_anterior"
carpeta_imagenes_2 = "/content/drive/MyDrive/imagenes_pruebas_Angie/imagenes_salida/choco_posterior"

In [9]:
# @title Análisis de coberturas **TOTAL**
import os
import re
import numpy as np
import rasterio
import pandas as pd



# Tamaño del píxel en metros
PIXEL_SIZE = 10
HECTARE_CONVERSION = 10000

# Función para agrupar imágenes
def agrupar_imagenes(carpeta):
    archivos = [f for f in os.listdir(carpeta) if f.endswith(".tiff")]
    patrones = {}
    for archivo in archivos:
        match = re.match(r"(.+?)_(.+?)_(\d{4}-\d{2}-\d{2})_((?:.+?)*)_(.+?)_(.+?).tiff", archivo)
        if match:
            departamento, shape, fecha, municipio, satelite, banda = match.groups()
            clave = f"{departamento}_{shape}_{fecha}_{municipio}"
            # Normalizar nombres de bandas (B011 -> B11)
            banda = banda.replace("B011", "B11")
            if clave not in patrones:
                patrones[clave] = {}
            patrones[clave][banda] = os.path.join(carpeta, archivo)
    return patrones

# Función para procesar imágenes
def process_image(bandas):
    required_bands = ['B08', 'B04', 'B11', 'B03']
    if not all(banda in bandas for banda in required_bands):
        print(f"Faltan bandas en el conjunto: {bandas.keys()}")
        return None, None, None
    with rasterio.open(bandas['B08']) as src:
        banda_NIR = src.read(1).astype('float32')
    with rasterio.open(bandas['B04']) as src:
        banda_roja = src.read(1).astype('float32')
    with rasterio.open(bandas['B11']) as src:
        banda_SWIR = src.read(1).astype('float32')
    with rasterio.open(bandas['B03']) as src:
        banda_verde = src.read(1).astype('float32')
    mascara = (banda_roja > 0) & (banda_NIR > 0) & (banda_SWIR > 0) & (banda_verde > 0)
    ndvi = (banda_NIR - banda_roja) / (banda_NIR + banda_roja)
    ndwi = (banda_verde - banda_SWIR) / (banda_verde + banda_SWIR)
    return ndvi, ndwi, mascara

# Función para clasificar la cobertura
def clasificar_cobertura(ndvi, ndwi):
    clasificacion = np.zeros_like(ndvi, dtype=int)
    clasificacion[((ndvi >= 0.7) & (ndvi <= 1))] = 1  # Manglar con degradación baja
    clasificacion[((ndvi >= 0.4) & (ndvi <= 0.6))] = 2  # Manglar con degradación media
    clasificacion[((ndvi >= 0.2) & (ndvi <= 0.3))] = 3  # Manglar con degradación alta
    clasificacion[((ndvi < 0.1) & (ndvi > -0.05))] = 4  # Sin vegetación
    clasificacion[(ndwi > 0.3)] = 5  # Agua y otros
    return clasificacion

# Función para calcular el área en hectáreas
def calcular_area_hectareas(cobertura):
    area_m2 = np.sum(cobertura) * (PIXEL_SIZE ** 2)
    return area_m2 / HECTARE_CONVERSION

# Agrupar imágenes de ambas carpetas
imagenes_agrupadas_1 = agrupar_imagenes(carpeta_imagenes_1)
imagenes_agrupadas_2 = agrupar_imagenes(carpeta_imagenes_2)

# Combinar imágenes de ambas carpetas
imagenes_agrupadas = {**imagenes_agrupadas_1, **imagenes_agrupadas_2}

# Lista para almacenar resultados temporales
resultados_temporales = []

# Procesar cada conjunto de imágenes
for clave, bandas in imagenes_agrupadas.items():
    if len(bandas) == 4:  # Solo procesamos grupos completos
        match = re.match(r"([^_]+)_([^_]+)_(\d{4}-\d{2}-\d{2})_([^_]+)", clave)
        if match:
            departamento, shape, fecha, municipio = match.groups()
        else:
            print(f"No se pudo procesar: {clave}")
            continue  # Saltar si la clave no cumple con el formato
        # Procesar imágenes
        ndvi, ndwi, mascara = process_image(bandas)
        if ndvi is None:
            continue  # Saltar conjuntos incompletos
        clasificacion = clasificar_cobertura(ndvi, ndwi)
        # Crear un ciclo para registrar cada cobertura por separado
        coberturas = {
            1: "Manglar con degradación baja",
            2: "Manglar con degradación media",
            3: "Manglar con degradación alta",
            4: "Sin vegetación",
            5: "Agua y otros"
        }
        # Calcular biomasa y área para cada cobertura
        for cobertura_id, cobertura_nombre in coberturas.items():
            cobertura_mask = clasificacion == cobertura_id
            # Calcular biomasa por cobertura
            biomasa = np.zeros_like(ndvi, dtype='float32')
            if cobertura_id in [1, 2, 3]:  # Solo para manglares
                biomasa[cobertura_mask] = -2.3 + 11.03 * ndvi[cobertura_mask]
            # Calcular área en hectáreas
            area_cobertura = calcular_area_hectareas(cobertura_mask)
            biomasa_total = biomasa[cobertura_mask].sum()
            # Calcular promedio NDVI
            promedio_ndvi = np.nanmean(ndvi[cobertura_mask]) if np.any(cobertura_mask) else 0
            # Registrar los resultados temporales
            resultados_temporales.append([
                departamento, municipio, float(shape), fecha, "Manglar", cobertura_nombre,
                area_cobertura,  # Superficie (ha)
                biomasa_total,  # Biomasa Total
                promedio_ndvi  # Promedio NDVI
            ])

# Convertir resultados temporales a DataFrame
df_temporal = pd.DataFrame(resultados_temporales, columns=[
    "Depto", "Municipio", "Shape_Leng", "Fecha", "Ecosistema", "Cobertura",
    "Superficie(ha)", "Biomasa Total", "Promedio NDVI"
])

# Calcular totales por Shape_Leng y Fecha
df_temporal["Total Área"] = df_temporal.groupby(["Shape_Leng", "Fecha"])["Superficie(ha)"].transform("sum")
df_temporal["Total Biomasa"] = df_temporal.groupby(["Shape_Leng", "Fecha"])["Biomasa Total"].transform("sum")
df_temporal["Total NDVI"] = df_temporal.groupby(["Shape_Leng", "Fecha"])["Promedio NDVI"].transform("sum")

# Calcular %Cobertura, %Biomasa y %NDVI
df_temporal["%Cobertura"] = (
    (df_temporal["Superficie(ha)"] / df_temporal["Total Área"]) * 100
    if df_temporal["Total Área"].iloc[0] > 0 else 0
)
df_temporal["%Biomasa"] = (
    (df_temporal["Biomasa Total"] / df_temporal["Total Biomasa"]) * 100
    if df_temporal["Total Biomasa"].iloc[0] > 0 else 0
)
df_temporal["%NDVI"] = (
    (df_temporal["Promedio NDVI"] / df_temporal["Total NDVI"]) * 100
    if df_temporal["Total NDVI"].iloc[0] > 0 else 0
)

# Reemplazar NaN por 0 en Promedio NDVI y %NDVI
df_temporal["%Cobertura"] = df_temporal["%Cobertura"].fillna(0)
df_temporal["%Biomasa"] = df_temporal["%Biomasa"].fillna(0)
df_temporal["Promedio NDVI"] = df_temporal["Promedio NDVI"].fillna(0)
df_temporal["%NDVI"] = df_temporal["%NDVI"].fillna(0)

# Seleccionar columnas finales
df_resultados = df_temporal[[
    "Depto", "Municipio", "Shape_Leng", "Fecha", "Ecosistema", "Cobertura",
    "Superficie(ha)", "%Cobertura", "Biomasa Total", "%Biomasa", "Promedio NDVI", "%NDVI"
]]

# Ordenar el DataFrame por Shape_Leng
df_resultados = df_resultados.sort_values(by="Shape_Leng", ascending=True)


<ipython-input-9-aed24937e690>:45: RuntimeWarning: invalid value encountered in divide
  ndvi = (banda_NIR - banda_roja) / (banda_NIR + banda_roja)
<ipython-input-9-aed24937e690>:46: RuntimeWarning: invalid value encountered in divide
  ndwi = (banda_verde - banda_SWIR) / (banda_verde + banda_SWIR)


In [10]:
df_resultados

,Depto,Municipio,Shape_Leng,Fecha,Ecosistema,Cobertura,Superficie(ha),%Cobertura,Biomasa Total,%Biomasa,Promedio NDVI,%NDVI
2864,Choco,El Litoral Del San Juan,0.498073,2024-02-25,Manglar,Agua y otros,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2862,Choco,El Litoral Del San Juan,0.498073,2024-02-25,Manglar,Manglar con degradación alta,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2861,Choco,El Litoral Del San Juan,0.498073,2024-02-25,Manglar,Manglar con degradación media,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2860,Choco,El Litoral Del San Juan,0.498073,2024-02-25,Manglar,Manglar con degradación baja,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1309,Choco,El Litoral Del San Juan,0.498073,2019-07-16,Manglar,Agua y otros,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2422,Choco,Bajo Baudo,4831.359279,2024-05-30,Manglar,Manglar con degradación alta,3.76,7.282588,197.040833,0.583739,0.256033,14.856611
2423,Choco,Bajo Baudo,4831.359279,2024-05-30,Manglar,Sin vegetación,0.18,0.348635,0.000000,0.000000,0.092983,5.395456
2424,Choco,Bajo Baudo,4831.359279,2024-05-30,Manglar,Agua y otros,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
872,Choco,Bajo Baudo,4831.359279,2021-07-05,Manglar,Manglar con degradación alta,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


# Escribe en Google Sheet

In [ ]:
# @title Escribe en Sheet **Final**
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from gspread_dataframe import set_with_dataframe

#Autenticación en Google Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Nombre de la hoja de Google Sheets donde se guardará el DataFrame
nombre_hoja = "Datos_Coberturas"

# Convertir valores no compatibles a formatos válidos
df_resultados.replace([np.inf, -np.inf], np.nan, inplace=True)  # Reemplaza `inf` por `NaN`
df_resultados.fillna("", inplace=True)  # Sustituye `NaN` por valores vacíos

# Convertir solo las columnas de texto a `str`, manteniendo los números como están
for col in df_resultados.select_dtypes(include=['object']).columns:
    df_resultados[col] = df_resultados[col].astype(str)

# Limitar el tamaño máximo de cada celda a 50,000 caracteres
df_resultados = df_resultados.applymap(lambda x: x[:50000] if isinstance(x, str) else x)

# Crear o abrir la hoja en Google Sheets
try:
    spreadsheet = gc.open(nombre_hoja)  # Intentar abrir la hoja existente
except gspread.exceptions.SpreadsheetNotFound:
    spreadsheet = gc.create(nombre_hoja)  # Si no existe, crear una nueva

# Seleccionar la primera hoja, si no existe, crearla
worksheet = spreadsheet.sheet1

# Limpiar la hoja antes de escribir nuevos datos
worksheet.clear()

# Escribir el DataFrame en Google Sheets sin perder formato numérico
set_with_dataframe(worksheet, df_resultados, include_index=False, include_column_header=True)

print(f"✅ Datos guardados correctamente en Google Sheets: {spreadsheet.url}")
